我们希望hbase里有完整的对应文章的数据，后面如果数据发生更新，只需要更新hbase的数据即可
HBase在设计上，family不宜过多，同时对于文章表我们构建一个hive的外接表
需要保证hbase和hive的thrift服务是启动的：
${HBASE_HOME}/bin/hbase-daemon.sh start thrift -threadpool
${HIVE_HOME}/bin/hive --service hiveserver2

In [26]:
class Conf:
    pass

conf = Conf()
conf.mongo_host = "localhost:27017"
conf.mongo_db = "resys-one-test"
conf.mongo_collection = "hb_articles"
conf.hbase_db = "videobase_all"
conf.hive_db = "resys_one"
conf.hive_table = conf.hbase_db
conf.__dict__

{'mongo_host': 'localhost:27017',
 'mongo_db': 'resys-one-test',
 'mongo_collection': 'hb_articles',
 'hbase_db': 'videobase_all',
 'hive_db': 'resys_one',
 'hive_table': 'videobase_all'}

In [61]:
# 首先构建这个hbase数据库和对应的hive外接表
import happybase
conn = happybase.Connection('localhost', autoconnect=True)
conn.open()
table_list = conn.tables()
table_list

[b'hive_test_hbase',
 b'spark',
 b'test',
 b'user_list_click',
 b'videobase_all',
 b'zzti']

happybase需要hbase配置这几项，否则会断：
<property>
         <name>hbase.thrift.server.socket.read.timeout</name>
         <value>600000</value>
</property>
<property>
         <name>hbase.thrift.connection.max-idletime</name>
         <value>600000</value>
</property>
设置为10分钟

In [11]:
if conf.hbase_db not in table_list:
    conn.create_table(conf.hbase_db, {'d': dict(max_versions=1, block_cache_enabled=False)})
table = conn.table(conf.hbase_db)
table.families()

{b'd': {'name': b'd:',
  'max_versions': 1,
  'compression': b'NONE',
  'in_memory': False,
  'bloom_filter_type': b'NONE',
  'bloom_filter_vector_size': 0,
  'bloom_filter_nb_hashes': 0,
  'block_cache_enabled': False,
  'time_to_live': 2147483647}}

In [67]:
# 创建这个hbase库对应的hive表
from pyhive import hive
cursor = hive.connect("localhost").cursor()
cursor

In [69]:
# 如果需要的话，先删了之前的表
cursor.execute("drop table if exists {0}.{1}".format(conf.hive_db, conf.hive_table))

In [70]:
# 创建外链表，为了便于后续操作，hive中保存的都是字符串，至于字段含义，可以后续进行处理
cursor.execute("create database if not exists " + conf.hive_db )
external_sql="""
 create external table if not exists {0}.{1} (
    id string,
    title string,
    content string,
    state string,
    createTime string,
    updateTime string,
    publishTime string,
    source string,
    articleAuthor string,
    categorys string,
    clickNum string,
    qualityScore string)
 stored by 'org.apache.hadoop.hive.hbase.HBaseStorageHandler'
 WITH SERDEPROPERTIES ("hbase.columns.mapping" = "d:title,
                                                  d:content,
                                                  d:state,
                                                  d:createTime,
                                                  d:updateTime,
                                                  d:publishTime,
                                                  d:source,
                                                  d:articleAuthor,
                                                  d:categorys,
                                                  d:clickNum,
                                                  d:qualityScore")
 TBLPROPERTIES("hbase.table.name" = "{2}")
""".format(conf.hive_db, conf.hive_table, conf.hbase_db)
print(external_sql)
cursor.execute(external_sql)


 create external table if not exists resys_one.videobase_all (
    id string,
    title string,
    content string,
    state string,
    createTime string,
    updateTime string,
    publishTime string,
    source string,
    articleAuthor string,
    categorys string,
    clickNum string,
    qualityScore string)
 STORED BY 'org.apache.hadoop.hive.hbase.HBaseStorageHandler'
 WITH SERDEPROPERTIES ("hbase.columns.mapping" = "d:title,
                                                  d:content,
                                                  d:state,
                                                  d:createTime,
                                                  d:updateTime,
                                                  d:publishTime,
                                                  d:source,
                                                  d:articleAuthor,
                                                  d:categorys,
                                                  d:clickN

In [31]:
# 将mongo中的文章写入hbase
import pymongo
mgconn = pymongo.MongoClient("mongodb://localhost:27017/")
articles = mgconn[conf.mongo_db][conf.mongo_collection].find()
articles = [ a for a in articles ]
len(articles)

1000

In [34]:
articles[0]

{'_id': '470308283',
 'title': '借鉴：这篇最受欢迎校训，没有一个字讲学习',
 'content': '“我知道，我不是因为偶然才来到这个世界，我是为了践行一个平凡、美丽、无私的梦想而来的；我是为了通过各种苦乐逆顺的体验来历练自己而来的，并由此完善，成长而提升……”金华市环城小学校训，没有一个字和学习有关，却让学生、家长和老师都掉下泪来。校训原文： 世界因我多温暖 我知道，我不是因为偶然才来到这个世界，我是为了践行一个平凡、美丽、无私的梦想而来的;我是为了通过各种苦乐逆顺的体验来历练自己而来的，并由此完善，成长而提升。 我深深地知道，改变这个世界的力量来自太阳，来自人类心灵深处的温度。我，要让世界因我而多温暖。 我知道，我所有的长处都源自父母祖宗的优秀，源自华夏千年文明的积淀。但它不是我炫耀和自私的资本，它是我赖以成长并服务人类的工具，它是我生命的伟大、美好和无私的工具。 我知道，我的缺点与不足不是我的自愿，那是因为我是从有缺点和不足的爸爸妈妈而来，选择这样的爸爸妈妈是我的自愿。对于这些缺陷，我全然接受，并通过今生的感恩、忍受和努力来弥补。 我想对爸爸妈妈说，我愿意从今天开始，不再用完美要求你们，也请你们不再用完美苛求于我，我是你们的一部分，我们是一个整体，让我们一起改变，用爱让家里充满温暖，以影响世界。 从今天起，我将高高地放飞自己的梦想，积极乐观地生活和学习。 命运从来没有规定我此生将是什么?国家没有规定我，父母没有规定我，老师也是一样。一切万物都没有规定我必须是什么样的人，大家把一切主动权交给我，让我自己决定自己的梦想，然后慈悲无私地帮助我，成就我。 因此，我必须让我自己成为一颗最圆润的种子，让周边的世界因我的成长而温暖。 我知道，生命是人世间最美丽的奇迹，读书是人世间最享受的愉悦。 老师对我说，曾经有一个善人，在春天的时候特别给两个乞丐一间破房和一块空地。到了秋天，一个懒惰的乞丐贫病而死，而另一个勤奋的乞丐却富裕安乐。 在宇宙中，每一个灵魂都是乞丐，四处漂泊。父母就是善人，给了属于我的一间破房和广袤无垠的空地，那间破房就是我不完美的身体，而那块空地就是我无边的心灵。我坚信，只要用勤劳播撒智慧与爱的种子，就一定会有硕果累累的明天。 从这一刻起，我要用无限的信心走向未来。 我知道，生命中最珍贵最强大的就是灵魂。环城小学是我人生的第

In [58]:
articles_to_hbase = [ {
    "d:id": a['_id'],
    "d:title":a['title'],
    "d:content":a['content'],
    "d:state":str(a['state']),
    "d:createTime":a['createTime'].strftime('%Y-%m-%d-%H-%M-%S'),
    "d:updateTime":a['updateTime'].strftime('%Y-%m-%d-%H-%M-%S'),
    "d:publishTime":a['publishTime'].strftime('%Y-%m-%d-%H-%M-%S'),
    "d:source":a['source'],
    "d:articleAuthor":a['articleAuthor'],
    "d:categorys":",".join(a['categorys']),
    "d:clickNum":str(a['clickNum']),
    "d:qualityScore":str(a['qualityScore'])
} for a in articles ]
articles_to_hbase[0]

{'d:id': '470308283',
 'd:title': '借鉴：这篇最受欢迎校训，没有一个字讲学习',
 'd:content': '“我知道，我不是因为偶然才来到这个世界，我是为了践行一个平凡、美丽、无私的梦想而来的；我是为了通过各种苦乐逆顺的体验来历练自己而来的，并由此完善，成长而提升……”金华市环城小学校训，没有一个字和学习有关，却让学生、家长和老师都掉下泪来。校训原文： 世界因我多温暖 我知道，我不是因为偶然才来到这个世界，我是为了践行一个平凡、美丽、无私的梦想而来的;我是为了通过各种苦乐逆顺的体验来历练自己而来的，并由此完善，成长而提升。 我深深地知道，改变这个世界的力量来自太阳，来自人类心灵深处的温度。我，要让世界因我而多温暖。 我知道，我所有的长处都源自父母祖宗的优秀，源自华夏千年文明的积淀。但它不是我炫耀和自私的资本，它是我赖以成长并服务人类的工具，它是我生命的伟大、美好和无私的工具。 我知道，我的缺点与不足不是我的自愿，那是因为我是从有缺点和不足的爸爸妈妈而来，选择这样的爸爸妈妈是我的自愿。对于这些缺陷，我全然接受，并通过今生的感恩、忍受和努力来弥补。 我想对爸爸妈妈说，我愿意从今天开始，不再用完美要求你们，也请你们不再用完美苛求于我，我是你们的一部分，我们是一个整体，让我们一起改变，用爱让家里充满温暖，以影响世界。 从今天起，我将高高地放飞自己的梦想，积极乐观地生活和学习。 命运从来没有规定我此生将是什么?国家没有规定我，父母没有规定我，老师也是一样。一切万物都没有规定我必须是什么样的人，大家把一切主动权交给我，让我自己决定自己的梦想，然后慈悲无私地帮助我，成就我。 因此，我必须让我自己成为一颗最圆润的种子，让周边的世界因我的成长而温暖。 我知道，生命是人世间最美丽的奇迹，读书是人世间最享受的愉悦。 老师对我说，曾经有一个善人，在春天的时候特别给两个乞丐一间破房和一块空地。到了秋天，一个懒惰的乞丐贫病而死，而另一个勤奋的乞丐却富裕安乐。 在宇宙中，每一个灵魂都是乞丐，四处漂泊。父母就是善人，给了属于我的一间破房和广袤无垠的空地，那间破房就是我不完美的身体，而那块空地就是我无边的心灵。我坚信，只要用勤劳播撒智慧与爱的种子，就一定会有硕果累累的明天。 从这一刻起，我要用无限的信心走向未来。 我知道，生命中最珍贵最强大的就是灵魂。环城小学是

In [56]:
# 从mongo拿到的时间是datatime.datatime，需要将其转为时间戳
import datetime
now = datetime.datetime.now()
now.strftime('%Y-%m-%d-%H-%M-%S')

'2021-12-10-17-26-46'

In [62]:
table = conn.table(conf.hbase_db)
table.families()

{b'd': {'name': b'd:',
  'max_versions': 1,
  'compression': b'NONE',
  'in_memory': False,
  'bloom_filter_type': b'NONE',
  'bloom_filter_vector_size': 0,
  'bloom_filter_nb_hashes': 0,
  'block_cache_enabled': False,
  'time_to_live': 2147483647}}

In [63]:
for article in articles_to_hbase:
    table.put(row=article["d:id"],
              data=article,
              timestamp=1,
              wal=False)
print(table.row(row="470880100"))

{b'd:articleAuthor': b'zhangsan', b'd:categorys': b'\xe6\x9d\xa5\xe6\xba\x90-\xe6\xb5\x8b\xe8\xaf\x95', b'd:clickNum': b'0', b'd:content': b'\xe4\xb8\xad\xe5\x9b\xbd\xe5\x8f\xa4\xe4\xbb\xa3\xe7\x8e\xba\xe5\x8d\xb0\xe7\x9a\x84\xe5\x87\xba\xe7\x8e\xb0\xef\xbc\x8c\xe5\x8f\xaf\xe4\xbb\xa5\xe4\xb8\x8a\xe6\xba\xaf\xe8\x87\xb3\xe5\x95\x86\xe4\xbb\xa3\xe3\x80\x82\xe5\x8d\xb0\xe8\x80\x85\xef\xbc\x8c\xe4\xbf\xa1\xe4\xb9\x9f\xef\xbc\x8c\xe5\x8d\xb0\xe7\xab\xa0\xe7\x9a\x84\xe4\xb8\xbb\xe8\xa6\x81\xe5\x8a\x9f\xe8\x83\xbd\xe5\xb0\xb1\xe6\x98\xaf\xe7\xa4\xba\xe4\xbf\xa1\xef\xbc\x8c\xe9\x9a\x8f\xe7\x9d\x80\xe4\xbd\xa9\xe5\x8d\xb0\xe9\xa3\x8e\xe6\xb0\x94\xe7\x9a\x84\xe5\xbd\xa2\xe6\x88\x90\xef\xbc\x8c\xe5\x90\x89\xe8\xaf\xad\xe5\x8d\xb0\xe3\x80\x81\xe8\x82\x96\xe5\xbd\xa2\xe5\x8d\xb0\xe7\xad\x89\xe5\xbd\xb0\xe6\x98\xbe\xe5\xae\xa1\xe7\xbe\x8e\xe5\x8a\x9f\xe8\x83\xbd\xe7\x9a\x84\xe5\xa4\x9a\xe7\xa7\x8d\xe5\x8d\xb0\xe5\xbc\x8f\xe9\x80\x90\xe6\xb8\x90\xe7\x9b\x9b\xe8\xa1\x8c\xe3\x80\x82\xe5\x94\x90\xe5\xa

In [64]:
# 使用hive查一下试试，注意这样做会启动一个MR任务
cursor.execute("select count(*) from {0}.{1}".format(conf.hive_db, conf.hive_table))

In [65]:
for row in cursor.fetchall():
    print(row)

(1000,)


In [66]:
# 关闭刚才建立的连接
conn.close()
cursor.close()
mgconn.close()
